Models were automatically reconstructed from NCBI RefSeq accession codes with carveme.

For M5:

```
carve --refseq GCF_002238085.1 --gapfill m5_med -i m5_med --mediadb medium.tsv -o GEMs/M5.xml 
```

Tried w/o gapfill - this model could not grow on the supplied medium.


In [1]:
from cobra.io import read_sbml_model
from utils import model_validation as mv

m5 = read_sbml_model('GEMs/M5.xml')

In [25]:
m5_nogapfill = read_sbml_model('GEMs/M5_nogapfill.xml')

In [26]:
medium = m5.medium
m5_nogapfill.medium = medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [33]:
m5_nogapfill.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# Testing the model:

1. check whether growth is possible on the medium,  
use ```growth_possible```.

1. check whether the exhange reactions exist in the model,  
use ```reactions_exist```.

2. check whether the exhange reactions are blocked,  
use ```check_blocked_reactions```.

2. check whether the exhange reactions can carry flux at optimal solution (FVA),  
use ```check_production```.

In [3]:
mv.growth_possible(m5)

True

In [10]:
m5.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg__L_e,EX_arg__L_e,10,6,7.28%
asp__L_e,EX_asp__L_e,1.745,4,0.85%
ca2_e,EX_ca2_e,0.007358,0,0.00%
cl_e,EX_cl_e,0.007358,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001414,0,0.00%
cu2_e,EX_cu2_e,0.001002,0,0.00%
cys__L_e,EX_cys__L_e,0.1303,3,0.05%
dcyt_e,EX_dcyt_e,10,9,10.92%
fe2_e,EX_fe2_e,1.149,0,0.00%
fe3_e,EX_fe3_e,0.01104,0,0.00%


In [4]:
uptake_reactions = [
    'EX_xyl__D_e', # xylose exchange
]

production_reactions = [
    'EX_btoh_e', # butanol exhange
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
    'EX_acetone_e', # acetone exchange
]

In [6]:
print(mv.reactions_exist(m5, uptake_reactions))
print(mv.reactions_exist(m5, production_reactions))

True
The following reactions are missing: ['EX_btoh_e', 'EX_acetone_e']
False


In [7]:
existing_production_reactions = [
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
]

mv.check_production(m5, existing_production_reactions)

,minimum,maximum
EX_etoh_e,0.0,0.0
EX_ac_e,0.0,0.0
EX_but_e,0.0,0.0


In [24]:
mv.check_blocked_reactions(m5, existing_production_reactions)

['EX_etoh_e', 'EX_ac_e', 'EX_but_e']

The out-of-the-box carveme model:
1. can grow on the medium  
and does use xylan (however it seems like xylan -> xylose conversion is happening intracellularly)
1. does not contain the exhange reactions for butanol or acetone
1. does not contain acetone, butyrate, or ethanol production in its solution space

## troubleshooting

In [19]:
m5.metabolites.xylan4_e.summary()

Percent,Flux,Reaction,Definition
100.00%,18.26,XYLe,h2o_e + xylan8_e --> 2.0 xylan4_e
Percent,Flux,Reaction,Definition
100.00%,-18.26,XYLANabc,atp_c + h2o_c + xylan4_e --> adp_c + h_c + pi_c + xylan4_c


In [34]:
metabolites = ['btoh', '1btol', 'btal', 'btcoa', 'butpi', 'but', 'acetone', 'acac', 'aacoa', 'b2coa', '3hbcoa', 'accoa', 'pyr']

for metabolite in metabolites:
    try:
        m5.metabolites.get_by_id(str(metabolite+"_c"))
    except KeyError:
        print(metabolite, "not in model")

butpi not in model
acetone not in model


In [36]:
reactions = ['HBCO_nadp','ACOAD1f', 'BTCOARx', 'ALCD4', 'BTS', 'PBUTT', 'BUTKr', 'BUTCT', 'BUTCT','EX_acetone_e', 'EX_but_e', 'EX_btoh_e']

for reaction in reactions:
    try:
        m5.reactions.get_by_id(reaction)
    except KeyError:
        print(reaction, "not in model")

PBUTT not in model
BUTKr not in model
EX_acetone_e not in model
EX_btoh_e not in model
